In [1]:
import pandas as pd

/Users/mahiradayal/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


"credit" shows the percentage of people who have a credit score under 660 by county in 2020 Q1. See creditscore_scrape.ipynb for methodology on how this data (Equifax Subprime Credit Population FRED Economic Data) was scraped and cleaned.

In [2]:
credit = pd.read_csv("/Users/mahiradayal/masters_project/full_credit.csv")

"banks" shows the banks per county (using FIP codes). See https://colab.research.google.com/drive/1AXF7Y1J829PHL6bQHsxSYZM1kk7gUn64?usp=sharing for methodology on how FDIC banks data was cleaned, combined with census income and population data, merged using FIP codes and how the banks per capita figure was reached. Previous notebooks show how this FDIC data was geocoded and tested using the Google Maps API. 

In [3]:
banks = pd.read_csv("/Users/mahiradayal/masters_project/bankscredit.csv")
banks = banks.drop(columns=['B01003_001M', 'B19013_001M', 'state', 'county'])
banks = banks.rename(columns={"Banks": "Number of Banks"})
banks.head()

,FIPS,Number of Banks,POPULATION,MEDIAN_HOUSEHOLD_INCOME,BANKS_PER_CAP*100,Name,State
0,29189,1,996919,67420,0.000100,St Louis,MO
1,17043,1,929060,92809,0.000108,Du Page,IL
2,24033,1,908670,84920,0.000110,Prince Georges,MD
3,13089,1,749323,62399,0.000133,De Kalb,GA
4,11001,1,692683,86420,0.000144,Washington,DC


In [4]:
credit.head()

,County Name,State,FIPS County Code,Link,url,q1_credit,credit_percent
0,AUTAUGA,AL,1001,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">30...",30.03273
1,BALDWIN,AL,1003,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">24...",24.19225
2,BARBOUR,AL,1005,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">41...",41.07486
3,BIBB,AL,1007,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">35...",35.78089
4,BLOUNT,AL,1009,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">27...",27.83199


In [5]:
# Looking at whether there's a correlation between number of banks and credit scores

column_1 = credit["credit_percent"]
column_2 = banks["Number of Banks"]
correlation = column_1.corr(column_2)

In [6]:
correlation

-0.02368768215815885

A negative correlation is useful because our credit value is "percentage of people with low credit scores" so a higher value here means a lower credit score. Taking this as a correlation of 0.024. 

In [7]:
pd.set_option('display.max_rows', 500)
louisiana = banks[banks.State == "LA"]

In [8]:
credit = credit.rename(columns={"FIPS County Code": "FIPS"})
credit['FIPS'] = credit['FIPS'].apply(pd.to_numeric)
louisiana['FIPS'] = louisiana['FIPS'].apply(pd.to_numeric)

<ipython-input-8-6436b3882ef5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  louisiana['FIPS'] = louisiana['FIPS'].apply(pd.to_numeric)


In [9]:
louisiana = louisiana.merge(credit, on='FIPS', how='left')

In [10]:
louisiana

,FIPS,Number of Banks,POPULATION,MEDIAN_HOUSEHOLD_INCOME,BANKS_PER_CAP*100,Name,State_x,County Name,State_y,Link,url,q1_credit,credit_percent
0,22103,1,255155,68905,0.000392,St Tammany,LA,ST. TAMMANY,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">24...",24.59031
1,22097,1,83172,36403,0.001202,St Landry,LA,ST. LANDRY,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">37...",37.34508
2,22099,1,53732,48656,0.001861,St Martin,LA,ST. MARTIN,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">34...",34.82838
3,22089,1,52773,69019,0.001895,St Charles,LA,ST. CHARLES,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">28...",28.14916
4,22101,1,50968,40485,0.001962,St Mary,LA,ST. MARY,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">37...",37.36462
5,22087,1,46266,44661,0.002161,St Bernard,LA,ST. BERNARD,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">38...",38.24724
6,22095,1,43242,57429,0.002313,St John the Baptist,LA,ST. JOHN BAPTIST,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">41...",41.78045
7,22093,1,21308,51603,0.004693,St James,LA,ST. JAMES,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">33...",33.48066
8,22091,1,10297,43886,0.009712,St Helena,LA,ST. HELENA,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">45...",45.48944
9,22063,22,138928,63389,0.015836,Livingston,LA,LIVINGSTON,LA,https://fred.stlouisfed.org/series/EQFXSUBPRIME0,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">32...",32.32078


In [11]:
# Data points for Caddo: 
# 245831 population 
# $41797 median household income
# 75 banks
# 36.5% of people have a credit score under 660

In [12]:
# Data points for Louisiana: 
louisiana['MEDIAN_HOUSEHOLD_INCOME'] = louisiana['MEDIAN_HOUSEHOLD_INCOME'].apply(pd.to_numeric)
louisiana['credit_percent'] = louisiana['credit_percent'].apply(pd.to_numeric)

In [13]:
# Average household income
louisiana['MEDIAN_HOUSEHOLD_INCOME'].mean()

44874.015625

In [14]:
# Average credit percentage
louisiana['credit_percent'].mean()

35.5480525

In [15]:
california = banks[banks.State == "CA"]

In [16]:
california = california.merge(credit, on='FIPS', how='left')

In [17]:
# Data points for Imperial County: 
# 180701 population 
# $47622 median household income
# 15 banks
# 30.0% of people have a credit score under 660

In [18]:
# Data points for California: 
california['MEDIAN_HOUSEHOLD_INCOME'] = california['MEDIAN_HOUSEHOLD_INCOME'].apply(pd.to_numeric)
california['credit_percent'] = california['credit_percent'].apply(pd.to_numeric)

In [19]:
# Average household income
california['MEDIAN_HOUSEHOLD_INCOME'].mean()

67719.20338983051

In [20]:
# Average credit percentage
california['credit_percent'].mean()

18.70469288135593

In [21]:
arizona = banks[banks.State == "AZ"]

In [22]:
arizona = arizona.merge(credit, on='FIPS', how='left')
arizona

,FIPS,Number of Banks,POPULATION,MEDIAN_HOUSEHOLD_INCOME,BANKS_PER_CAP*100,Name,State_x,County Name,State_y,Link,url,q1_credit,credit_percent
0,4001,5,71511,32508,0.006992,Apache,AZ,APACHE,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">49...",49.90602
1,4009,3,37996,51353,0.007896,Graham,AZ,GRAHAM,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">24...",24.65753
2,4021,37,432793,58174,0.008549,Pinal,AZ,PINAL,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">26...",26.54216
3,4011,1,9522,63473,0.010502,Greenlee,AZ,GREENLEE,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">26...",26.19808
4,4027,23,209468,45243,0.010980,Yuma,AZ,YUMA,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">23...",23.67285
5,4003,17,125867,49260,0.013506,Cochise,AZ,COCHISE,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">21...",21.87265
6,4005,20,141274,59460,0.014157,Coconino,AZ,COCONINO,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">25...",25.16038
7,4017,16,109270,40067,0.014643,Navajo,AZ,NAVAJO,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">33...",33.99582
8,4019,154,1027207,53379,0.014992,Pima,AZ,PIMA,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">22...",22.50870
9,4013,735,4328810,64468,0.016979,Maricopa,AZ,MARICOPA,AZ,https://fred.stlouisfed.org/series/EQFXSUBPRIME00,https://fred.stlouisfed.org/series/EQFXSUBPRIM...,"<span class=""series-meta-observation-value"">23...",23.27985


In [23]:
# Data points for Apache County: 
# 71511 population 
# $32508 median household income
# 5 banks
# 50.0% of people have a credit score under 660

In [24]:
# Data points for Arizona: 
arizona['MEDIAN_HOUSEHOLD_INCOME'] = arizona['MEDIAN_HOUSEHOLD_INCOME'].apply(pd.to_numeric)
arizona['credit_percent'] = arizona['credit_percent'].apply(pd.to_numeric)

In [25]:
# Average household income
arizona['MEDIAN_HOUSEHOLD_INCOME'].mean()

48989.933333333334

In [26]:
# Average credit percentage
arizona['credit_percent'].mean()

25.831815333333335